In [2]:
from selenium import webdriver      
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import urllib.request
import time

from datetime import date
import pandas as pd
date_today = date.today()

URL = 'https://vetementswebsite.com/product-category/clothing/'
LOAD_MORE_BUTTON_XPATH = '/html/body/div[2]/div/div[2]/div/div/main/div/div/div[3]/div[4]/a/span'
driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
driver.get(URL)

while True:
    try:
        loadMoreButton = driver.find_element_by_xpath("/html/body/div[2]/div/div[2]/div/div/main/div/div/div[3]/div[4]/a/span")
        time.sleep(3)
        loadMoreButton.click()
        time.sleep(5)
    except Exception as e:
        print(e)
        break

soup = BeautifulSoup(driver.page_source,'html.parser')


Message: element not interactable
  (Session info: chrome=87.0.4280.88)



In [16]:
name = []
product_url = []
picture_url = []
sales_price = []
price = []

timestamp = []
brand = []
product_type = []

for i in soup.find('div',{'class':'col-xs-12'}).find_all('li'):
    
    timestamp.append(date_today)
    brand.append('Vetements')
    product_type.append(None)
    
    name.append(i.find('div',{'class':'product_item--info'}).h3.text)
    product_url.append(i.find('div',{'class':'product_item--info'}).h3.a.get('href'))
    
    picture_url.append(i.find('div',{'class':'product_item--thumbnail'}).a.img.get('src'))
    sales_price.append(i.find('span',{'class':'price'}).text)
    price.append(None)

# Second Layer

In [27]:
material = []
product_description = []
picture_name = []
color = []
counter = 1

for link in product_url:

    ourUrl = urllib.request.urlopen(link)
    soup = BeautifulSoup(ourUrl,'html.parser')
    if soup.find('tr',{'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_color'}) != None:
        color.append(soup.find('tr',{'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_color'}).td.text.strip())
    else:
        color.append(None)
        
    if soup.find('tr',{'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_material'}) != None:
        material.append(soup.find('tr',{'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_material'}).td.text.strip())
    else:
        material.append(None)
    picture_name.append('undercover-women_' + str(counter) + '.jpg') # change variable
    counter += 1
    product_description.append(None)
    

In [28]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", "picture_name"]
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Vetements,None,Be Better Humans T-Shirt,None,None,€330,White,Cotton,https://vetementswebsite.com/shop/be-better-hu...,https://vetementswebsite.com/wp-content/upload...,2020-12-08,undercover-women_1.jpg
1,Vetements,None,Beige Coat,None,None,"€2,610",Brown,"Synthetic, Wool",https://vetementswebsite.com/shop/beige-coat/,https://vetementswebsite.com/wp-content/upload...,2020-12-08,undercover-women_2.jpg
2,Vetements,None,Black Cotton Hoodie,None,None,€780,Black,"Cotton, Synthetic",https://vetementswebsite.com/shop/black-cotton...,https://vetementswebsite.com/wp-content/upload...,2020-12-08,undercover-women_3.jpg
3,Vetements,None,Black Cotton Shirt,None,None,€650,Black,Cotton,https://vetementswebsite.com/shop/black-cotton...,https://vetementswebsite.com/wp-content/upload...,2020-12-08,undercover-women_4.jpg
4,Vetements,None,Black Leggings,None,None,€350,Black,Synthetic,https://vetementswebsite.com/shop/black-leggings/,https://vetementswebsite.com/wp-content/upload...,2020-12-08,undercover-women_5.jpg


In [29]:
second_pass.to_csv("vetements.csv", index=False, encoding = 'utf-8-sig')

# Download Images

In [30]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/vetements.csv', encoding='utf-8-sig')

os.mkdir('vetements')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('vetements/'+ "vetements_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1